In [1]:
# import accelerate

In [1]:
%env HF_ENDPOINT=https://hf-mirror.com
import os
os.environ['HF_HOME'] = '/data1/ckw'

env: HF_ENDPOINT=https://hf-mirror.com


In [2]:
os.environ['HF_ENDPOINT']='https://hf-mirror.com'

In [3]:
!pwd

/data1/ckw/01大语言模型/ChatGLM4


In [4]:
import torch
# from transformers import AutoModelForCausalLM, AutoTokenizer
from tokenization_chatglm import ChatGLM4Tokenizer
from modeling_chatglm import ChatGLMForConditionalGeneration

device = "cuda"

tokenizer = ChatGLM4Tokenizer.from_pretrained("THUDM/glm-4-9b-chat", trust_remote_code=True)

query = "你好"

inputs = tokenizer.apply_chat_template([{"role": "user", "content": query}],
                                       add_generation_prompt=True,
                                       tokenize=True,
                                       return_tensors="pt",
                                       return_dict=True
                                       )

inputs = inputs.to(device)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
model = ChatGLMForConditionalGeneration.from_pretrained(
    "THUDM/glm-4-9b-chat",
    torch_dtype=torch.bfloat16,
    low_cpu_mem_usage=True,
    trust_remote_code=True,
    load_in_4bit=True,
    device_map='auto'
).eval()#.to(device)

gen_kwargs = {"max_length": 2500, "do_sample": True, "top_k": 1}
with torch.no_grad():
    outputs = model.generate(**inputs, **gen_kwargs)
    outputs = outputs[:, inputs['input_ids'].shape[1]:]
    print(tokenizer.decode(outputs[0], skip_special_tokens=True))

The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/10 [00:00<?, ?it/s]


你好👋！有什么可以帮助你的吗？


In [5]:
from transformers.utils import is_accelerate_available

In [10]:
is_accelerate_available()

False

In [8]:
??is_accelerate_available

Signature: is_accelerate_available(min_version: str = '0.21.0')
Docstring: <no docstring>
Source:   
def is_accelerate_available(min_version: str = ACCELERATE_MIN_VERSION):
    return _accelerate_available and version.parse(_accelerate_version) >= version.parse(min_version)
File:      /data1/ckw/micromamba/envs/kewei-ai/lib/python3.11/site-packages/transformers/utils/import_utils.py
Type:      function

In [9]:
%pip install --upgrade accelerate

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 2.2 MB/s eta 0:00:001.8 MB/s eta 0:00:01
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.20.3
    Uninstalling accelerate-0.20.3:
      Successfully uninstalled accelerate-0.20.3
Note: you may need to restart the kernel to use updated packages.
